# DORC score calculation Ma et. al.

In [1]:
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from glob import glob

## Import cellxgene, cellxpeak matrix

In [2]:
gm = sc.read_h5ad('../archr/ArchROutput/filtered_gene_bc_matrix/hplacenta_gene_matrix.h5ad')
gm

AnnData object with n_obs × n_vars = 36456 × 36546
    obs: 'Sample', 'TSSEnrichment', 'ReadsInTSS', 'ReadsInPromoter', 'ReadsInBlacklist', 'PromoterRatio', 'PassQC', 'NucleosomeRatio', 'nMultiFrags', 'nMonoFrags', 'nFrags', 'nDiFrags', 'BlacklistRatio', 'Gex_nUMI', 'Gex_nGenes', 'Gex_MitoRatio', 'Gex_RiboRatio', 'DoubletScore', 'DoubletEnrichment', 'gest_week', 'Clusters', 'ReadsInPeaks', 'FRIP'
    obsm: 'X_lsi', 'X_umap'

In [3]:
pm = sc.read_h5ad('../archr/ArchROutput/filtered_peak_bc_matrix/hplacenta_peak_matrix.h5ad')
pm

AnnData object with n_obs × n_vars = 36456 × 444087
    obs: 'Sample', 'TSSEnrichment', 'ReadsInTSS', 'ReadsInPromoter', 'ReadsInBlacklist', 'PromoterRatio', 'PassQC', 'NucleosomeRatio', 'nMultiFrags', 'nMonoFrags', 'nFrags', 'nDiFrags', 'BlacklistRatio', 'Gex_nUMI', 'Gex_nGenes', 'Gex_MitoRatio', 'Gex_RiboRatio', 'DoubletScore', 'DoubletEnrichment', 'gest_week', 'Clusters', 'ReadsInPeaks', 'FRIP'
    var: 'seqnames', 'start', 'end', 'width', 'strand', 'score', 'replicateScoreQuantile', 'groupScoreQuantile', 'Reproducibility', 'GroupReplicate', 'distToGeneStart', 'nearestGene', 'peakType', 'distToTSS', 'nearestTSS', 'GC', 'idx', 'N'

## Import p2g data

In [4]:
'''p2g of all celltypes, 50kbp for gene filtering'''
# file = '../ArchR/p2g_50Kbp.tsv'
file = '../archr/ArchROutput/p2g/p2g_50Kbp_all.tsv'
p2g_f = pd.read_csv(file, sep='\t')
p2g_f.head()

,idxATAC,idxRNA,Correlation,FDR,VarQATAC,VarQRNA
0,7,18,0.597734,5.339842e-48,0.954592,0.828901
1,12,18,0.539746,1.374997e-37,0.786049,0.828901
2,44,24,0.569356,1.206013e-42,0.741163,0.558146
3,55,27,0.549368,3.550542e-39,0.992222,0.759344
4,56,27,0.455931,9.893002e-26,0.562496,0.759344


In [5]:
p2g_f

,idxATAC,idxRNA,Correlation,FDR,VarQATAC,VarQRNA
0,7,18,0.597734,5.339842e-48,0.954592,0.828901
1,12,18,0.539746,1.374997e-37,0.786049,0.828901
2,44,24,0.569356,1.206013e-42,0.741163,0.558146
3,55,27,0.549368,3.550542e-39,0.992222,0.759344
4,56,27,0.455931,9.893002e-26,0.562496,0.759344
...,...,...,...,...,...,...
43617,444019,36412,0.654730,1.498817e-60,0.958062,0.980053
43618,444021,36412,0.651682,8.242938e-60,0.747750,0.980053
43619,444023,36412,0.705643,2.537153e-74,0.954763,0.980053
43620,444027,36412,0.551602,1.493521e-39,0.700757,0.980053


In [6]:
## Find genes with more than 10 peaks
thresh = 10
mask_genes = np.zeros((len(np.unique(p2g_f.idxRNA))), dtype=bool)
for i,gid in enumerate(np.unique(p2g_f.idxRNA)):
    mask = (p2g_f.idxRNA == gid)
    
    mask_genes[i] = np.sum(mask) > thresh

#     df = pd.concat([df, pd.DataFrame(np.sum(pm_csr[:,p2g[mask].idxATAC-1], axis=1), columns=gm.var_names[gid-1])], axis=1)
#     df.loc[:,gm.var_names[gid-1]] = np.sum(pm_csr[:,p2g[mask].idxATAC-1], axis=1)

mask_genes

array([False, False, False, ..., False, False, False])

In [7]:
'''normalize peak matrix'''
pm.raw = pm
pm.X = pm.X.multiply(1/np.sum(pm.X,axis=1)).multiply(1e6)
np.sum(pm.X, axis=1)

matrix([[1000000.06],
        [1000000.  ],
        [ 999999.94],
        ...,
        [1000000.  ],
        [ 999999.94],
        [ 999999.94]], dtype=float32)

In [8]:
'''p2g of all celltypes'''
# data_root = '../data/WOT/'
file = '../archr/ArchROutput/p2g/p2g_500Kbp_all.tsv'
# file = '../archr/ArchROutput/p2g/p2g_1Mbp_all.tsv'

In [ ]:
p2g = pd.read_csv(file, sep='\t')

## Calculate DORC scores
pm_csr = pm.X.tocsr()

gene_ids = np.unique(p2g_f.idxRNA)[mask_genes]
gene_names = gm.var_names[gene_ids-1]

dorc = np.zeros((pm_csr.shape[0], len(gene_ids)))
peak_sum = np.zeros(len(gene_ids))
for i,gid in enumerate(gene_ids):
    mask = p2g.idxRNA == gid

    dorc[:,i] = np.sum(pm_csr[:,p2g[mask].idxATAC-1], axis=1).flatten()
    peak_sum[i] = len(p2g[mask])

In [11]:
df = pd.DataFrame(dorc, index = pm.obs_names, columns=gene_names)
df_peaksum = pd.DataFrame(peak_sum, index = gene_names)

df

,HES4,MMP23B,PRKCZ,GPR153,GPR157,SPSB1,TNFRSF1B,AADACL3,C1orf158,PDPN,...,EGFL7,TMSB4X,EGFL6,MIR222HG,MSN,OPHN1,SRPX2,TSC22D3,AMOT,VGLL1
JS34#ACGTCAAGTTGCAATG-1,185.543839,289.912231,162.350845,289.912231,81.175423,452.263062,115.964890,0.000000,0.000000,313.105225,...,104.368401,104.368401,382.684143,185.543823,115.964890,0.000000,46.385956,57.982445,0.000000,0.000000
JS34#TGTTGGCCAAGGTACG-1,163.067993,172.660233,239.805862,38.368938,134.291275,153.475754,47.961174,76.737877,57.553406,0.000000,...,95.922340,38.368938,38.368938,19.184469,9.592235,182.252457,0.000000,67.145645,67.145645,115.106812
JS34#TTTGCGACAGTACCGT-1,161.838470,118.681549,323.676971,43.156929,172.627701,140.260010,43.156929,107.892319,86.313858,10.789232,...,107.892326,43.156929,43.156929,53.946163,0.000000,21.578465,0.000000,118.681549,43.156929,43.156929
JS34#AACCTTGCACATAGCC-1,77.012680,154.025360,317.677338,48.132927,134.772186,202.158295,86.639267,96.265854,57.759510,19.253170,...,38.506340,38.506340,48.132927,38.506340,0.000000,192.531708,0.000000,96.265854,77.012680,96.265854
JS34#AGAGAGGAGGTGTCCA-1,164.486313,142.554810,54.828766,76.760277,142.554794,164.486313,230.280807,0.000000,0.000000,0.000000,...,581.184998,87.726028,43.863014,109.657532,109.657539,43.863014,65.794525,54.828766,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JS36#CATGGATTCCAAGTGT-1,594.353638,594.353638,0.000000,594.353638,0.000000,0.000000,594.353638,0.000000,0.000000,1188.707275,...,594.353638,594.353638,0.000000,594.353638,0.000000,0.000000,297.176819,0.000000,0.000000,0.000000
JS36#CGCACAATCATAACTG-1,0.000000,0.000000,1264.622192,0.000000,632.311096,0.000000,0.000000,0.000000,0.000000,0.000000,...,316.155548,0.000000,0.000000,0.000000,0.000000,632.311096,0.000000,0.000000,0.000000,0.000000
JS36#GCTAGCCAGACCATAC-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
JS36#CCAAACTAGTTTGGGT-1,0.000000,597.907349,896.861023,0.000000,0.000000,597.907349,597.907349,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [12]:
df_peaksum

,0
HES4,39.0
MMP23B,51.0
PRKCZ,45.0
GPR153,55.0
GPR157,26.0
...,...
OPHN1,27.0
SRPX2,19.0
TSC22D3,19.0
AMOT,22.0


In [15]:
df_peaksum.sort_values(by=0, ascending=False).to_csv(str(file)[:-4]+'_peaks_thresh'+str(thresh)+'_sum_sorted.tsv', sep='\t', header=None)

In [ ]:
df.to_csv(str(file)[:-4]+'_dorc_thresh'+str(thresh)+'.tsv', sep='\t')